# Load Data

# Dim Customer

In [1]:
%%sql
INSERT INTO core.dim_customer(
  customer_id, customer_city, customer_state, customer_zip_code_prefix, customer_geolocation_lat, customer_geolocation_lng)
  
(SELECT customer_id, customer_city, customer_state, customer_zip_code_prefix, geolocation_lat, geolocation_lng

FROM staging.customers

INNER JOIN staging.geolocation ON geolocation.geolocation_zip_code_prefix = customers.customer_zip_code_prefix)

UsageError: Cell magic `%%sql` not found.


# Dim Product

In [2]:
%%sql
INSERT INTO core.dim_product(product_id, product_category, product_category_english)

(SELECT product_id, products.product_category_name, product_category_name_english

FROM staging.products

LEFT JOIN staging.product_category_name_translation ON product_category_name_translation.product_category_name = products.product_category_name)

UsageError: Cell magic `%%sql` not found.


# Dim Seller

In [3]:
%%sql
INSERT INTO core.dim_seller(seller_id, seller_city, seller_state, seller_zip_code_prefix, seller_geolocation_lon, seller_geolocation_lat)

(SELECT seller_id, seller_city, seller_state, seller_zip_code_prefix, geolocation_lng, geolocation_lat

FROM staging.sellers

LEFT JOIN staging.geolocation ON geolocation.geolocation_zip_code_prefix = sellers.seller_zip_code_prefix)

UsageError: Cell magic `%%sql` not found.


# Dim Date

In [4]:
%%sql
INSERT INTO core.dim_date("date","year","month","monthname","day","dayofyear","weekdayname","calendarweek",
  							"formatteddate","quartal","yearquartal","yearmonth","yearcalendarweek", "monthstart","monthend")
 (SELECT
	--dwh_date_id,
	datum as Date,
	extract(year from "datum") AS Year,
	extract(month from "datum") AS Month,
	-- Localized month name
	to_char("date", 'TMMonth') AS MonthName,
	extract(day from datum) AS Day,
	extract(doy from datum) AS DayOfYear,
	-- Localized weekday
	to_char("date", 'TMDay') AS WeekdayName,
	-- ISO calendar week
	extract(week from datum) AS CalendarWeek,
	to_char(datum, 'dd. mm. yyyy') AS FormattedDate,
	'Q' || to_char(datum, 'Q') AS Quartal,
	to_char("date", 'yyyy/"Q"Q') AS YearQuartal,
	to_char("date", 'yyyy/mm') AS YearMonth,
	-- ISO calendar year and week
	to_char("date", 'iyyy/IW') AS YearCalendarWeek,

	-- Start and end of the month of this date
	"datum" + (1 - extract(day from datum))::integer AS MonthStart,
	("datum" + (1 - extract(day from datum))::integer + '1 month'::interval)::date - '1 day'::interval AS MonthEnd
FROM (
	-- There are 3 leap years in this range, so calculate 365 * 10 + 3 records
	SELECT SEQUENCE.day + 1 dwh_date_id, '2016-09-15'::DATE + sequence.day AS "datum"
	FROM generate_series(0,2190) AS sequence(day)
	GROUP BY sequence.day
     ) DQ
order by 1);

UsageError: Cell magic `%%sql` not found.


# Dim Time

In [5]:
%%sql
INSERT INTO core.dim_time(dwh_time_id,timeofday,hour,quarterhour,minute,minute_of_day,daytimename,daynight)
(SELECT 
	dwh_time_id,
	to_char(minute, 'hh24:mi') AS TimeOfDay,
	-- Hour of the day (0 - 23)
	extract(hour from minute) as Hour, 
	-- Extract and format quarter hours
	to_char(minute - (extract(minute from minute)::integer % 15 || 'minutes')::interval, 'hh24:mi') ||
	' – ' ||
	to_char(minute - (extract(minute from minute)::integer % 15 || 'minutes')::interval + '14 minutes'::interval, 'hh24:mi')
		as QuarterHour,
	EXTRACT(MINUTE FROM MINUTE)::INTEGER AS "minute",
	-- Minute of the day (0 - 1439)
	extract(hour from minute)*60 + extract(minute from minute) as minute_of_day,
	-- Names of day periods
	case when to_char(minute, 'hh24:mi') between '06:00' and '08:29'
		then 'Morning'
	     when to_char(minute, 'hh24:mi') between '08:30' and '11:59'
		then 'AM'
	     when to_char(minute, 'hh24:mi') between '12:00' and '17:59'
		then 'PM'
	     when to_char(minute, 'hh24:mi') between '18:00' and '22:29'
		then 'Evening'
	     else 'Night'
	end as DaytimeName,
	-- Indicator of day or night
	case when to_char(minute, 'hh24:mi') between '07:00' and '19:59' then 'Day'
	     else 'Night'
	end AS DayNight
from (SELECT SEQUENCE.minute +1 dwh_time_id, '0:00'::time + (sequence.minute || ' minutes')::interval AS minute
	FROM generate_series(0,1439) AS sequence(minute)
	GROUP BY sequence.minute
     ) DQ
order by 1)

UsageError: Cell magic `%%sql` not found.


# F_Orders

In [6]:
%%sql
INSERT INTO core.f_orders(dwh_date_id, dwh_time_id, order_id, dwh_product_id, dwh_seller_id, dwh_customer_id, order_product_price, oder_product_amount)

WITH facts AS 

(SELECT order_items.order_id, orders.order_approved_at, orders.customer_id, order_items.product_id, order_items.seller_id, price, freight_value, 1 "amount"

FROM staging.order_items

LEFT JOIN staging.orders ON orders.order_id = order_items.order_id
  
WHERE orders.order_approved_at IS NOT NULL
  	AND order_status NOT IN ('unavailable', 'canceled')
)

SELECT dwh_date_id, dwh_time_id, order_id, dwh_product_id, dwh_seller_id, dwh_customer_id, price order_product_price, freight_value order_product_freight_value, amount order_product_amount

FROM facts


LEFT JOIN core.dim_customer ON dim_customer.customer_id = facts.customer_id

LEFT JOIN core.dim_product ON dim_product.product_id = facts.product_id 

LEFT JOIN core.dim_seller ON dim_seller.seller_id = facts.seller_id

LEFT JOIN core.dim_date ON (dim_date.day = EXTRACT(DAY FROM facts.order_approved_at)
  							AND dim_date.month = EXTRACT(MONTH FROM facts.order_approved_at)
  							AND dim_date.year = EXTRACT(YEAR FROM facts.order_approved_at))
                            
LEFT JOIN core.dim_time ON (dim_time.hour = EXTRACT(HOUR from facts.order_approved_at)
  							AND dim_time.minute = EXTRACT(minute FROM facts.order_approved_at))

UsageError: Cell magic `%%sql` not found.
